<a href="https://colab.research.google.com/github/anakstei/esp32-tinyml-boolean-logic/blob/main/esp32_tinyml_boolean_logic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras import Input
from tensorflow.data import Dataset
import numpy as np

In [2]:
# Function for generating dataset
def data_generator():
  while(True):
    a = np.random.randint(2)
    b = np.random.randint(2)
    X = [a, b]
    y = a ^ b
    yield X, [y]

In [3]:
# Generate dataset
train_dataset = tf.data.Dataset.from_generator(data_generator, output_types = (tf.int32, tf.int32), output_shapes=((2), (1)))
train_dataset = train_dataset.batch(batch_size=25)

In [4]:
# Create a neural network model
model = Sequential([Input(shape=(2)), Dense(6, activation='relu'), Dense(1, activation='sigmoid')])
model.compile(optimizer='adam', loss=tf.keras.losses.BinaryCrossentropy(), metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 6)                 18        
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 7         
Total params: 25
Trainable params: 25
Non-trainable params: 0
_________________________________________________________________


In [5]:
# Train the model
model.fit(train_dataset, steps_per_epoch=1000, epochs=4)

Epoch 1/4
1000/1000 [==============================] - 7s 6ms/step - loss: 0.6904 - accuracy: 0.4851
Epoch 2/4
1000/1000 [==============================] - 6s 6ms/step - loss: 0.3739 - accuracy: 1.0000
Epoch 3/4
1000/1000 [==============================] - 6s 6ms/step - loss: 0.1707 - accuracy: 1.0000
Epoch 4/4
1000/1000 [==============================] - 6s 6ms/step - loss: 0.0780 - accuracy: 1.0000


In [6]:
# Test the model
X_test = np.array([
    [0, 0],
    [0, 1],
    [1, 0],
    [1, 1]])
y_test = model.predict_on_batch(X_test)

In [7]:
print(y_test)

[[0.14213994]
 [0.9871634 ]
 [0.9829788 ]
 [0.00666034]]


In [8]:
print(np.round(y_test))

[[0.]
 [1.]
 [1.]
 [0.]]


In [9]:
# Mount google drive
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [10]:
# Convert the model to tflite model
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
def representative_dataset_gen():
  for _ in range(10000):
    yield [np.array([np.random.uniform(), np.random.uniform()], dtype=np.float32)]
converter.representative_dataset = representative_dataset_gen
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
tflite_quant_model = converter.convert()
open("/gdrive/MyDrive/Colab Notebooks/esp32-tinyml-boolean-logic/converted_model.tflite", "wb").write(tflite_quant_model)

INFO:tensorflow:Assets written to: /tmp/tmpctvkb8tu/assets


2192

In [11]:
# Install tool for converting the tflite model to hexadecimal
!apt-get install xxd

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following NEW packages will be installed:
  xxd
0 upgraded, 1 newly installed, 0 to remove and 30 not upgraded.
Need to get 49.3 kB of archives.
After this operation, 200 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 xxd amd64 2:8.0.1453-1ubuntu1.4 [49.3 kB]
Fetched 49.3 kB in 0s (139 kB/s)
Selecting previously unselected package xxd.
(Reading database ... 160980 files and directories currently installed.)
Preparing to unpack .../xxd_2%3a8.0.1453-1ubuntu1.4_amd64.deb ...
Unpacking xxd (2:8.0.1453-1ubuntu1.4) ...
Setting up xxd (2:8.0.1453-1ubuntu1.4) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...


In [12]:
# Convert the tflite model to hexadecimal
!xxd -i "/gdrive/MyDrive/Colab Notebooks/esp32-tinyml-boolean-logic/converted_model.tflite" > "/gdrive/MyDrive/Colab Notebooks/esp32-tinyml-boolean-logic/model_data.cc"